FastCosine은 sparse matrix에 대하여 Cosine similarity를 이용하였을 때 유사한 점들 (k-NN, r-NN)을 찾기 위한 인덱서이다. 

In [1]:
import sys

sys.path.append('../../../soy')
from soy.ml.neighbors import FastCosine

scikit.feature_extractor.text.CountVectorizer 등에 의하여 만들어진 matrix market (mm) 파일로부터 인덱싱을 한다

mm 파일은 만들어졌다고 가정하며, 인덱싱이 끝나면 FastCosine을 저장할 모델 파일도 미리 설정해두자

In [2]:
mm_file = ''      # Fill your file
model_prefix = '' # Fill your file

### Indexing

FastCosine을 이용하기 위해서는 .indexing(mm_file)을 넣으면 된다. 

인덱싱 과정은 네 단계로 나뉘어진다. 

1. mm 파일을 읽어들여 임시로 이용할 term-document matrix를 (simple inverted index) 메모리에 올린다.
1. 두 벡터의 Cosine similarity는 유닛 벡터로 정규화된 두 벡터의 내적의 곱으로 계산할 수 있기 때문에, document 기준으로 term vector를 유닛 벡터화 한다. 
1. 모든 term에 대하여 term: (weight tuple, doc num tuple, doc id tuple) 세 종류로 구분된 champion list를 만든다. 
    
        inverted[term] = ( 
          (w1, w2, w3),
          (num_doc_w1, num_doc_w2, num_doc_w3),
          ((d11, d12), (d21, d22, d23, d24), (d31, d32))
        ) 
   
1. query processing에서 어떤 term부터 살펴볼지 순서를 결정하기 위하여 inverted document frequency 값을 계산하여 모델에 저장한다. Search term order는 반드시 idf가 아닐 수도 있기 때문에 term_order 표현한다.

각 네 단계가 끝나면 아래와 같은 메시지가 출력된다. 

    loaded mm
    normalized t2d weight
    builded champion list
    computed search term order (idf)

In [3]:
fast_cosine = FastCosine()
fast_cosine.indexing(mm_file)

loaded mm
normalized t2d weight
builded champion list
computed search term order (idf)


### Saving

모델 저장은 .save(model_prefix)으로 할 수 있으며, champion list 파일과 idf가 각각 따로 저장된다. prefix에 다른 tail을 붙이고 싶다면 아래의 함수들을 모두 각각 실행하면 된다. 

    def save(self, model_prefix):
        self._save_inverted_index('%s_inverted_index' % model_prefix)
        self._save_idf('%s_idf' % model_prefix)

In [4]:
fast_cosine.save(model_prefix)

### Loading

반대로 모델을 부르는 것은 .load(model_prefix)로 할 수 있다. 이 역시 champion list와 idf를 각각 직접 불러올 수 있다. 

    def load(self, model_prefix):
        self._load_inverted_index('%s_inverted_index' % model_prefix)
        self._load_idf('%s_idf' % model_prefix)

In [5]:
loaded_fast_cosine = FastCosine()
loaded_fast_cosine.load(model_prefix)

### k-Nearest neighbor search

query는 {term:weight} 형식의 dictionary 형태여야 한다. 유닛 벡터로 정규화 되어 있으면 좋지만, 그렇지 않아도 계산의 영향력은 없다. 인덱싱된 다른 문서들이 모두 유닛 벡터화 되어있기 때문에 내적값의 순위와 Cosine의 순위가 같기 때문이다. 하지만 earlystop_cut에 영향을 주기 때문에 query 마다 다른 earlystop_cut을 넣어야 한다. 그렇지 않으려면 쿼리 역시 유닛 벡터로 만들면 된다. 

Parameters는 다음과 같다. 

    query: {term id: weight}
    n_neighbors: 검색할 최인접 이웃의 개수, default 10
    candidate_factor: 최인접 이웃을 계산할 때, 각 term 마다 고려하는 이웃 후보의 최대 배수. 
                      3이면 각 term 마다 30개의 문서를 후보로 이용함. default 3.0
    remain_tfidf_threshold: query term weight * idf가 threshold 보다 작으면 종료. 
                            default 1.0
    max_weight_factor: champion list에서 각 term 별로 weight 최대값보다 
                       factor 배 이하인 문서들은 후보에서 제외. default 0.5, 
    scoring_by_adding: True면 내적 대신 덧셈, default True
    compute_true_cosine: True면 계산된 최인접이웃들에 대하여 실제 Cosine을 계산. default False
    
Return은 두 가지 neighbors와 info가 된다. 

    neighbors: [(doc_id, sim), (doc_id, sim), ...]과 같은 list of tuple 형식
    info: 검색과 관련된 메타 정보가 출력된다. 

info에는 다음과 같은 정보가 저장되어 있다. 
    
    calculated_percentage: query term의 몇 퍼센트를 계산에 이용했는지이다. 
                           0.84라면 다른 문서에서 최대 0.16만큼 코싸인이 증가할 수 있다.
    n_candidate: k-NN을 찾기 위한 실제 후보들의 개수
    n_computation: k-NN을 찾는 과정에서 이뤄진 덧셈, 혹은 곱셈을 수행한 횟수
    n_considered_terms: query term에서 계산에 사용된 term의 개수
    n_terms_in_query: query term의 개수 
    time [mil.sec]: 각 process 마다 소모된 시간
        check_query_type: query term 중 인덱싱 된 term만 고르는 과정
        order_search_term: query term 중 어떤 term부터 고려해야 하는지 순서를 정하는 과정
        retrieval_similars: 이웃을 추정하는 과정
        true_cosine_computation: 실제 Cosine을 계산하는 과정
        whole_querying_process: kneighbors를 실행하는 총 과정

In [11]:
query={1:0.3, 10:0.2, 783:0.1, 3322:0.1}
query = {t:v/sum(query.values()) for t,v in query.items()}

neighbors, info = fast_cosine.kneighbors(query, compute_true_cosine=True,\
                                         remain_tfidf_threshold=0.2, \
                                         max_weight_factor=0.2,\
                                         scoring_by_adding=False)

from pprint import pprint
pprint(neighbors)
pprint(info)

[(3157868, 0.316227766016838),
 (306731, 0.27386127875258304),
 (189501, 0.24494897427831783),
 (1040234, 0.2335496832484569),
 (2299947, 0.2335496832484569),
 (2262923, 0.23094010767585033),
 (1850652, 0.22360679774997902),
 (1407105, 0.20701966780270628),
 (1987483, 0.20701966780270628),
 (3183462, 0.20701966780270628)]
{'calculated_percentage': 0.93333333333333346,
 'n_candidate': 85,
 'n_computation': 85,
 'n_considered_terms': 3,
 'n_terms_in_query': 4,
 'time [mil.sec]': {'check_query_type': 0.046253204345703125,
                    'order_search_term': 0.026941299438476562,
                    'retrieval_similars': 0.3027915954589844,
                    'true_cosine_computation': 0.39005279541015625,
                    'whole_querying_process': 0.7660388946533203}}


### Range search

range search는 인터페이스만 있으며, 아직 구현하지는 않았다. 

query, query_range=0.2, candidate_factor=3.0, earlystop_cut=1.0, w_cut=0.5, score_as_add=True, compute_true_cosine=False)

Parameters는 다음과 같다. 

    query_range: 검색할 최인접 이웃의 범위, Cosine distance. default 0.2
    
그 외의 parameters와 return 값은 동일하다

In [8]:
fast_cosine.rneighbors(query)

NotImplementedError: 